# Mello ML Pipeline - Complete Demonstration

This notebook demonstrates the complete Mello ML pipeline using the new object-oriented architecture.

## Features Demonstrated:
- 🧑‍💻 Creating and managing individual users
- 📚 Generating synthetic user data with AI
- 💭 Generating taste profiles from book preferences
- 🔢 Creating embeddings from taste descriptions
- 👥 Managing populations of users
- 📊 Applying UMAP, t-SNE, and PCA dimensionality reduction
- 📈 Interactive visualization of user personalities
- 🔍 Finding similar users and adding new users to populations
- 💾 Saving and loading data

## Setup and Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import json
from pathlib import Path

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Import our custom classes
from user import User, UserPreferences, BookRating
from population import Population
from profile_generator import ProfileGenerator
from embedding_generator import EmbeddingGenerator
from visualizer import Visualizer

print("✅ All imports successful!")
print("🚀 Ready to demonstrate the Mello ML pipeline")

/Users/yahyarahhawi/miniforge3/envs/ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All imports successful!
🚀 Ready to demonstrate the Mello ML pipeline


## 1. Creating Individual Users

Let's start by creating a few individual users manually to understand the User class.

In [3]:
# Create a new user
user1 = User(name="Alice Johnson")

# Add some books
user1.add_book("The Bell Jar", "Sylvia Plath", 5.0, "Fiction")
user1.add_book("Norwegian Wood", "Haruki Murakami", 4.5, "Fiction")
user1.add_book("The Catcher in the Rye", "J.D. Salinger", 4.0, "Fiction")
user1.add_book("Sapiens", "Yuval Noah Harari", 4.5, "Non-fiction")
user1.add_book("The Power of Now", "Eckhart Tolle", 3.5, "Self-help")

print(user1)
print("\nBook summary:")
print(user1.get_books_summary())

User(name='Alice Johnson', books=5, taste_profile=No, embedding=No, special=False)

Book summary:
Alice Johnson's book preferences:
- The Bell Jar by Sylvia Plath: 5.0/5 stars
- Norwegian Wood by Haruki Murakami: 4.5/5 stars
- The Catcher in the Rye by J.D. Salinger: 4.0/5 stars
- Sapiens by Yuval Noah Harari: 4.5/5 stars
- The Power of Now by Eckhart Tolle: 3.5/5 stars



In [4]:
# Create another user with different preferences
user2 = User(name="Bob Smith")

user2.add_book("Dune", "Frank Herbert", 5.0, "Science Fiction")
user2.add_book("1984", "George Orwell", 4.5, "Dystopian")
user2.add_book("The Foundation", "Isaac Asimov", 4.5, "Science Fiction")
user2.add_book("Thinking, Fast and Slow", "Daniel Kahneman", 4.0, "Psychology")
user2.add_book("The Art of War", "Sun Tzu", 3.5, "Philosophy")

print(user2)
print("\nBook summary:")
print(user2.get_books_summary())

User(name='Bob Smith', books=5, taste_profile=No, embedding=No, special=False)

Book summary:
Bob Smith's book preferences:
- Dune by Frank Herbert: 5.0/5 stars
- 1984 by George Orwell: 4.5/5 stars
- The Foundation by Isaac Asimov: 4.5/5 stars
- Thinking, Fast and Slow by Daniel Kahneman: 4.0/5 stars
- The Art of War by Sun Tzu: 3.5/5 stars



## 2. Generating Synthetic Users with AI

Now let's use the ProfileGenerator to create realistic synthetic users.

In [5]:
# Initialize the profile generator
profile_gen = ProfileGenerator()

print(f"Profile Generator: {profile_gen}")
print(f"Available personality archetypes: {len(profile_gen.personality_archetypes)}")
print("\nSome personality types:")
for i, archetype in enumerate(profile_gen.personality_archetypes[:5]):
    print(f"  {i+1}. {archetype}")

Profile Generator: ProfileGenerator(requests=0, archetypes=50)
Available personality archetypes: 50

Some personality types:
  1. Academic philosopher who loves ancient wisdom and existential questions
  2. Tech entrepreneur fascinated by science fiction and innovation
  3. Environmental activist drawn to nature writing and climate literature
  4. History buff obsessed with biographical accounts and war narratives
  5. Psychology student interested in human behavior and self-help


In [6]:
# Generate a single synthetic user
print("🤖 Generating a synthetic user...")
synthetic_user = profile_gen.generate_synthetic_user("introspective literary enthusiast")

if synthetic_user:
    print(f"\n✅ Generated: {synthetic_user}")
    print(f"📚 Number of books: {len(synthetic_user.preferences.books)}")
    print(f"🎭 Personality type: {synthetic_user.metadata.get('personality_type', 'Unknown')}")
    
    print("\n📖 First 5 books:")
    for i, book in enumerate(synthetic_user.preferences.books[:5]):
        print(f"  {i+1}. {book.title} by {book.author} - {book.rating}/5 ({book.genre})")
else:
    print("❌ Failed to generate synthetic user")

🤖 Generating a synthetic user...


2025-08-15 01:57:21,269 - INFO - Generated synthetic user: Gina Aguirre (introspective literary enthusiast)



✅ Generated: User(name='Gina Aguirre', books=20, taste_profile=No, embedding=No, special=False)
📚 Number of books: 20
🎭 Personality type: introspective literary enthusiast

📖 First 5 books:
  1. The Brothers Karamazov by Fyodor Dostoevsky - 5.0/5 (Fiction)
  2. Leaves of Grass by Walt Whitman - 4.8/5 (Poetry)
  3. Man's Search for Meaning by Viktor Frankl - 4.7/5 (Non-fiction (Psychology))
  4. The Master and Margarita by Mikhail Bulgakov - 4.6/5 (Fiction)
  5. Walden by Henry David Thoreau - 4.5/5 (Non-fiction (Nature/Philosophy))


## 3. Generating Taste Profiles

Let's generate AI-powered personality descriptions based on book preferences.

In [7]:
# Generate taste profile for our manually created user
print("🧠 Generating taste profile for Alice...")
success = profile_gen.generate_taste_profile(user1)

if success:
    print("\n✅ Taste Profile Generated!")
    print(f"\n📝 Alice's Taste Profile:")
    print(f"{user1.taste_profile}")
else:
    print("❌ Failed to generate taste profile")

🧠 Generating taste profile for Alice...


2025-08-15 01:57:39,166 - INFO - Generated taste profile for Alice Johnson



✅ Taste Profile Generated!

📝 Alice's Taste Profile:
This person exhibits a thoughtful and introspective nature, demonstrated by their literary preferences spanning fiction, non-fiction, and self-help. They appreciate narratives exploring individual experiences and emotions, suggesting an empathetic and emotionally intelligent approach to understanding the world. A preference for intellectual non-fiction indicates a curious and analytical mind, drawn to understanding broader societal and historical contexts. A potential interest in personal growth and self-improvement is also evident. They likely possess a balanced perspective, blending emotional sensitivity with analytical thinking. Their decision-making probably leans towards well-reasoned choices, incorporating both personal values and objective information. While appreciative of fictional works, they show ambition for deeper understanding with their non-fiction selections. Socially, they may display carefulness and insight,


In [8]:
# Generate taste profile for the synthetic user
if synthetic_user:
    print("🧠 Generating taste profile for synthetic user...")
    success = profile_gen.generate_taste_profile(synthetic_user)
    
    if success:
        print(f"\n✅ Taste Profile Generated for {synthetic_user.name}!")
        print(f"\n📝 {synthetic_user.name}'s Taste Profile:")
        print(f"{synthetic_user.taste_profile}")
    else:
        print("❌ Failed to generate taste profile")

🧠 Generating taste profile for synthetic user...


2025-08-15 01:57:55,422 - INFO - Generated taste profile for Gina Aguirre



✅ Taste Profile Generated for Gina Aguirre!

📝 Gina Aguirre's Taste Profile:
This person possesses a thoughtful and introspective nature, demonstrated by a clear preference for both profound literary works and philosophical explorations. They are drawn to narratives that delve into the complexities of the human condition, valuing emotional depth and intellectual stimulation in their reading choices. Highly valuing meaning and purpose, they demonstrate a keen interest in existential themes and the search for understanding in both personal and universal contexts. This indicates their empathetic and reflective spirit, likely leading them to engage compassionately with others, valuing authenticity and holding meaningful dialogue. Further, the blend of fiction and non-fiction demonstrates a curious mind that seeks balanced perspectives and deep learning, blending emotional understanding with a firm commitment to intellectual inquiry.


## 4. Generating Embeddings

Now let's convert taste profiles into vector embeddings.

In [9]:
# Initialize the embedding generator
pop = Population()
pop.load_from_json("demo_population.json")
embedding_gen = EmbeddingGenerator()

print(f"Embedding Generator: {embedding_gen}")
print(f"Model: {embedding_gen.model_name}")
print(f"Dimensions: {embedding_gen.embedding_dimensions}")

2025-08-15 01:57:57,019 - WARNING - User with name 'Yahya Rahhawi' already exists in population
2025-08-15 01:57:57,020 - WARNING - User with name 'Yahya Rahhawi' already exists in population
2025-08-15 01:57:57,020 - INFO - Loaded population 'Generated Population (200 users)' with 209 users


Embedding Generator: EmbeddingGenerator(provider=gemini, model=gemini-embedding-001, dims=3072, requests=0)
Model: gemini-embedding-001
Dimensions: 3072


In [10]:
# Generate embedding for Alice
if user1.taste_profile:
    print("🔢 Generating embedding for Alice...")
    success = embedding_gen.embed_user(user1)
    
    if success:
        print("✅ Embedding generated!")
        print(f"📊 Embedding shape: {user1.embedding.shape}")
        print(f"📊 First 10 values: {user1.embedding[:10]}")
    else:
        print("❌ Failed to generate embedding")
else:
    print("⚠️ Alice needs a taste profile first")

🔢 Generating embedding for Alice...


2025-08-15 01:58:02,384 - INFO - Generated embedding for user Alice Johnson


✅ Embedding generated!
📊 Embedding shape: (3072,)
📊 First 10 values: [-0.00460049 -0.02884815  0.01740649 -0.05061968 -0.00018011 -0.00686283
 -0.00341175  0.0148003  -0.00543704 -0.01034395]


In [11]:
# Test embedding a custom text
test_description = "They love philosophical novels that explore existential themes and enjoy books that challenge conventional thinking."

print(f"🔢 Generating embedding for test description...")
print(f"📝 Description: {test_description}")

try:
    test_embedding = embedding_gen.embed_text(test_description)
    print(f"\n✅ Embedding generated!")
    print(f"📊 Shape: {test_embedding.shape}")
    print(f"📊 First 10 values: {test_embedding[:10]}")
except Exception as e:
    print(f"❌ Error: {e}")

🔢 Generating embedding for test description...
📝 Description: They love philosophical novels that explore existential themes and enjoy books that challenge conventional thinking.

✅ Embedding generated!
📊 Shape: (3072,)
📊 First 10 values: [ 3.57252940e-05 -3.16982940e-02  2.54228300e-02 -4.99543920e-02
  8.85931300e-03  1.03857440e-02 -1.59768240e-02 -1.96687800e-03
  1.54259745e-02 -8.40698600e-03]


## 5. Creating and Managing Populations

Let's create a population and add multiple users to it.

In [12]:
# Create a new population
population = Population("Demo Population")

# Add our manually created users
population.add_user(user1)
population.add_user(user2)

# Add the synthetic user if it exists
if synthetic_user:
    population.add_user(synthetic_user)

print(f"Population: {population}")
print(f"\n📊 Population Statistics:")
stats = population.get_statistics()
for key, value in stats.items():
    print(f"  {key}: {value}")

Population: Population(name='Demo Population', users=3)

📊 Population Statistics:
  total_users: 3
  special_users: 0
  regular_users: 3
  users_with_embeddings: 1
  users_with_taste_profiles: 2
  total_books_rated: 30
  avg_books_per_user: 10.0
  embedding_dimension: 768
  avg_rating: 4.4366666666666665
  min_rating: 3.5
  max_rating: 5.0
  rating_std: 0.40123420702737805


In [12]:
# Generate a small population of synthetic users
print("🏭 Generating a population of 10 synthetic users...")

def progress_callback(current, total):
    print(f"Progress: {current}/{total} users generated")

synthetic_population = profile_gen.generate_population(size=200, progress_callback=progress_callback)

print(f"\n✅ Generated population: {synthetic_population}")
print(f"\n📊 Population Statistics:")
stats = synthetic_population.get_statistics()
for key, value in stats.items():
    print(f"  {key}: {value}")

2025-08-15 01:58:48,291 - INFO - Generating 200 synthetic users...


🏭 Generating a population of 10 synthetic users...


2025-08-15 01:58:53,128 - INFO - Generated synthetic user: Travis Williams (Minimalist drawn to simplicity, productivity, and intentional living)


Progress: 1/200 users generated


2025-08-15 01:58:58,724 - INFO - Generated synthetic user: James Carroll (Chef passionate about food culture, recipes, and culinary history)


...

Progress: 198/200 users generated


2025-08-15 02:16:31,741 - INFO - Generated synthetic user: Nicole Cooper (Social justice advocate reading about inequality and activism)


Progress: 199/200 users generated


2025-08-15 02:16:37,448 - INFO - Generated synthetic user: Catherine Dunn (Working parent seeking practical advice and quick fiction escapes)
2025-08-15 02:16:37,449 - INFO - Generated 200/200 users
2025-08-15 02:16:37,449 - INFO - Successfully generated 200/200 users


Progress: 200/200 users generated

✅ Generated population: Population(name='Generated Population (200 users)', users=200)

📊 Population Statistics:
  total_users: 200
  special_users: 0
  regular_users: 200
  users_with_embeddings: 0
  users_with_taste_profiles: 0
  total_books_rated: 4000
  avg_books_per_user: 20.0
  embedding_dimension: None
  avg_rating: 4.337325000000001
  min_rating: 1.0
  max_rating: 5.0
  rating_std: 0.5755383083470639


In [13]:
# Generate taste profiles for the entire population
print("🧠 Generating taste profiles for the population...")

successful_profiles = profile_gen.generate_taste_profiles_for_population(
    synthetic_population, 
    progress_callback=progress_callback
)

print(f"\n✅ Generated {successful_profiles} taste profiles")

# Show a few examples
print("\n📝 Sample taste profiles:")
for i, user in enumerate(synthetic_population.users[:3]):
    if user.taste_profile:
        print(f"\n{i+1}. {user.name}:")
        print(f"   {user.taste_profile[:100]}...")

2025-08-15 02:16:37,490 - INFO - Generating taste profiles for 200 users


🧠 Generating taste profiles for the population...


2025-08-15 02:16:40,961 - INFO - Generated taste profile for Travis Williams


Progress: 1/200 users generated


2025-08-15 02:16:44,137 - INFO - Generated taste profile for James Carroll


...

Progress: 198/200 users generated


2025-08-15 02:26:31,693 - INFO - Generated taste profile for Nicole Cooper


Progress: 199/200 users generated


2025-08-15 02:26:35,178 - INFO - Generated taste profile for Catherine Dunn
2025-08-15 02:26:35,179 - INFO - Successfully generated 200/200 taste profiles


Progress: 200/200 users generated

✅ Generated 200 taste profiles

📝 Sample taste profiles:

1. Travis Williams:
   This person is driven by a desire for self-improvement and a quest for a more meaningful and efficie...

2. James Carroll:
   This person possesses a curious and discerning mind, deeply engaged with the world of food and its b...

3. Denise Parker:
   This person is highly intellectual and deeply invested in understanding human behavior and potential...


## 6. Generating Embeddings for Population

Let's create embeddings for all users in our population.

In [14]:
# Generate embeddings for the entire population
print("🔢 Generating embeddings for the population...")

successful_embeddings = embedding_gen.embed_population(
    synthetic_population,
    progress_callback=progress_callback
)

print(f"\n✅ Generated {successful_embeddings} embeddings")

# Update population statistics
print(f"\n📊 Updated Population Statistics:")
stats = synthetic_population.get_statistics()
for key, value in stats.items():
    print(f"  {key}: {value}")

2025-08-15 02:26:35,218 - INFO - Generating embeddings for 200 users


🔢 Generating embeddings for the population...


2025-08-15 02:26:35,797 - INFO - Generated embedding for user Travis Williams


Progress: 1/200 users generated


2025-08-15 02:26:37,343 - INFO - Generated embedding for user James Carroll


...

Progress: 198/200 users generated


2025-08-15 03:02:23,644 - INFO - Generated embedding for user Nicole Cooper


Progress: 199/200 users generated


2025-08-15 03:02:25,090 - INFO - Generated embedding for user Catherine Dunn
2025-08-15 03:02:25,091 - INFO - Successfully generated 94/200 embeddings


Progress: 200/200 users generated

✅ Generated 94 embeddings

📊 Updated Population Statistics:
  total_users: 200
  special_users: 0
  regular_users: 200
  users_with_embeddings: 94
  users_with_taste_profiles: 200
  total_books_rated: 4000
  avg_books_per_user: 20.0
  embedding_dimension: 3072
  avg_rating: 4.337325000000001
  min_rating: 1.0
  max_rating: 5.0
  rating_std: 0.5755383083470639


In [ ]:
def add_user_to_population(population, name, taste_profile, special=False):
    """
    Add a user with name and taste profile to an existing population.
    Generates embedding automatically.
    
    Args:
        population: Existing Population object
        name: User's name
        taste_profile: User's taste profile description
        special: Boolean flag to mark user as special (default: False)
    
    Returns:
        User object if successful, None if failed
    """

    print(f"👤 Adding {name} to population{'⭐ (special user)' if special else ''}...")

    # Create new user with special flag
    user = User(name=name, special=special)

    # Set taste profile
    user.set_taste_profile(taste_profile)
    print(f"📝 Taste profile set ({len(taste_profile)} characters)")

    # Generate embedding
    print(f"🔢 Generating embedding...")
    if embedding_gen.embed_user(user):
        print(f"✅ Embedding generated! Shape: {user.embedding.shape}")

        # Add to population
        population.add_user(user)
        print(f"✅ {name} added to population{'⭐' if special else ''}!")
        print(f"📊 Population now has {len(population)} users")

        return user
    else:
        print(f"❌ Failed to generate embedding for {name}")
        return None

# Example usage - Add Einstein to existing population as special user
taste_profile = """
This individual exhibits a powerfully analytical and methodical cognitive style, characterized by rigorous observation, systematic experimentation, and an extraordinary capacity for mathematical abstraction. They are driven by an intense desire to uncover the underlying laws and mechanisms governing the natural world, demonstrating an unwavering persistence in their investigations. Emotionally, they appear to possess a detached and logical temperament, finding satisfaction in precision, order, and the verifiable proof of hypotheses. Their values are deeply rooted in empiricism, rational inquiry, and the pursuit of universal truths, prioritizing objective knowledge and demonstrable proof. Socially, they might appear somewhat solitary and intensely focused on their work, perhaps less concerned with interpersonal dynamics than with the grand intellectual challenges before them. Their decision-making is heavily reliant on logical deduction and a meticulous examination of evidence, leading to groundbreaking insights that fundamentally reshape scientific understanding.
"""
# Add Einstein to the existing population as a special user
user = add_user_to_population(synthetic_population, "Einstein", taste_profile, special=True)

👤 Adding Einstein to population⭐ (special user)...
📝 Taste profile set (1062 characters)
🔢 Generating embedding...


2025-08-15 11:34:50,525 - INFO - Generated embedding for user Einstein


✅ Embedding generated! Shape: (3072,)
✅ Einstein added to population⭐!
📊 Population now has 206 users


## 7. Applying Dimensionality Reduction (UMAP, t-SNE, PCA)

Now let's visualize our population using different dimensionality reduction techniques.

In [21]:
import plotly.graph_objects as go
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap

# Load population
# Get users with embeddings
users_with_embeddings = synthetic_population.get_users_with_embeddings()
embeddings = np.array([user.embedding for user in users_with_embeddings])
names = [user.name for user in users_with_embeddings]
profiles = [user.taste_profile or "No profile" for user in users_with_embeddings]
colors = ['orange' if user.special else 'blue' for user in users_with_embeddings]

# Standardize embeddings
scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)

# Fit PCA 3D
pca_3d = PCA(n_components=3, random_state=42)
pca_coords = pca_3d.fit_transform(embeddings_scaled)

# Fit UMAP 3D
umap_3d = umap.UMAP(n_components=3, n_neighbors=min(40, len(users_with_embeddings)-1),
                    min_dist=0.1, random_state=42)
umap_coords = umap_3d.fit_transform(embeddings_scaled)

# Create PCA plot
fig_pca = go.Figure()
fig_pca.add_trace(
    go.Scatter3d(
        x=pca_coords[:, 0],
        y=pca_coords[:, 1],
        z=pca_coords[:, 2],
        mode='markers',
        marker=dict(color=colors, size=8, opacity=0.8),
        text=names,
        customdata=profiles,
        hovertemplate='<b>%{text}</b><extra></extra>',
        name='Users'
    )
)

fig_pca.update_layout(
    title='Population Visualization: PCA 3D (Orange=Special, Blue=Regular)',
    scene=dict(
        xaxis_title='PCA Component 1',
        yaxis_title='PCA Component 2',
        zaxis_title='PCA Component 3'
    ),
    width=1200,
    height=800,
    showlegend=False
)

# Add click event handling for PCA
fig_pca.update_layout(
    annotations=[
        dict(
            text="Click on points to see full descriptions in browser console",
            showarrow=False,
            xref="paper", yref="paper",
            x=0.5, y=1.02, xanchor='center', yanchor='bottom'
        )
    ]
)

# Create UMAP plot
fig_umap = go.Figure()
fig_umap.add_trace(
    go.Scatter3d(
        x=umap_coords[:, 0],
        y=umap_coords[:, 1],
        z=umap_coords[:, 2],
        mode='markers',
        marker=dict(color=colors, size=8, opacity=0.8),
        text=names,
        customdata=profiles,
        hovertemplate='<b>%{text}</b><extra></extra>',
        name='Users'
    )
)

fig_umap.update_layout(
    title='Population Visualization: UMAP 3D (Orange=Special, Blue=Regular)',
    scene=dict(
        xaxis_title='UMAP Component 1',
        yaxis_title='UMAP Component 2',
        zaxis_title='UMAP Component 3'
    ),
    width=1200,
    height=800,
    showlegend=False
)

# Add click event handling for UMAP
fig_umap.update_layout(
    annotations=[
        dict(
            text="Click on points to see full descriptions in browser console",
            showarrow=False,
            xref="paper", yref="paper",
            x=0.5, y=1.02, xanchor='center', yanchor='bottom'
        )
    ]
)

# Save plots individually
fig_pca.write_html('population_pca_3d.html')
fig_umap.write_html('population_umap_3d.html')

# Show plots
print("📊 PCA 3D Plot:")
fig_pca.show()

print("\n📊 UMAP 3D Plot:")
fig_umap.show()

print(f"\n✅ Plots saved:")
print(f"   📁 population_pca_3d.html")
print(f"   📁 population_umap_3d.html")
print(f"\n💡 Hover over points to see names")
print(f"💡 Click on points to see full descriptions in browser console")

📊 PCA 3D Plot:


/Users/yahyarahhawi/miniforge3/envs/ml/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.




📊 UMAP 3D Plot:



✅ Plots saved:
   📁 population_pca_3d.html
   📁 population_umap_3d.html

💡 Hover over points to see names
💡 Click on points to see full descriptions in browser console


In [18]:
synthetic_population.users[-10:]

[User(name='Ryan Mason', books=20, taste_profile=Yes, embedding=Yes, special=False),
 User(name='Jean Phillips', books=20, taste_profile=Yes, embedding=Yes, special=False),
 User(name='Kimberly Russell', books=20, taste_profile=Yes, embedding=Yes, special=False),
 User(name='Anthony Kent', books=20, taste_profile=Yes, embedding=Yes, special=False),
 User(name='Tom Brown', books=20, taste_profile=Yes, embedding=Yes, special=False),
 User(name='Susan Gibson', books=20, taste_profile=Yes, embedding=Yes, special=False),
 User(name='Brianna Chandler', books=20, taste_profile=Yes, embedding=Yes, special=False),
 User(name='Nicole Cooper', books=20, taste_profile=Yes, embedding=Yes, special=False),
 User(name='Catherine Dunn', books=20, taste_profile=Yes, embedding=Yes, special=False),
 User(name='Yahya Rahhawi', books=0, taste_profile=Yes, embedding=Yes, special=True)⭐]

## 9. Finding Similar Users

Let's demonstrate similarity search functionality.

In [20]:
# Find similar users to one in our population
if len(users_with_embeddings) >= 3:
    target_user = synthetic_population.get_special_users()[0]
    print(f"🔍 Finding users similar to: {target_user.name}")
    print(f"📝 Target profile: {target_user.taste_profile[:100]}...")
    
    similar_users = synthetic_population.find_similar_users(target_user, top_k=3)
    
    print(f"\n👥 Top 3 most similar users:")
    for i, (user, similarity) in enumerate(similar_users):
        print(f"  {i+1}. {user.name} (similarity: {similarity:.3f})")
        print(f"     Profile: {user.taste_profile[:120]}...")
        print()

🔍 Finding users similar to: Yahya Rahhawi
📝 Target profile: 
This individual possesses a deeply introspective and philosophical mind, consistently seeking profo...

👥 Top 3 most similar users:
  1. Ryan Mason (similarity: 0.948)
     Profile: This individual possesses a deeply intellectual and philosophical nature, driven by a quest for understanding complex sy...

  2. Heather Green (similarity: 0.947)
     Profile: This person possesses a deeply intellectual and introspective nature, driven by a desire to understand the fundamental p...

  3. Heather Moore (similarity: 0.946)
     Profile: This individual possesses a deeply philosophical and introspective nature, constantly seeking meaning and understanding ...



In [37]:
# Search by custom embedding
if len(users_with_embeddings) >= 3:
    custom_description = "They enjoy thoughtful science fiction that explores philosophical questions about humanity and technology."
    print(f"🔍 Searching for users similar to: '{custom_description}'")
    
    try:
        # Generate embedding for the description
        custom_embedding = embedding_gen.embed_text(custom_description)
        
        # Find similar users
        similar_users = synthetic_population.find_similar_users_by_embedding(custom_embedding, top_k=3)
        
        print(f"\n👥 Top 3 most similar users:")
        for i, (user, similarity) in enumerate(similar_users):
            print(f"  {i+1}. {user.name} (similarity: {similarity:.3f})")
            print(f"     Books: {len(user.preferences.books)} rated")
            print(f"     Profile: {user.taste_profile[:80]}...")
            print()
    except Exception as e:
        print(f"❌ Error in similarity search: {e}")

🔍 Searching for users similar to: 'They enjoy thoughtful science fiction that explores philosophical questions about humanity and technology.'

👥 Top 3 most similar users:
  1. Nicholas Hardy (similarity: 0.897)
     Books: 20 rated
     Profile: They are deeply introspective and intellectually curious, with a strong interest...

  2. Christopher Kim (similarity: 0.888)
     Books: 20 rated
     Profile: They possess a thoughtful and introspective nature, drawn to works that explore ...

  3. Samuel Cruz (similarity: 0.888)
     Books: 20 rated
     Profile: They possess a deep intellectual curiosity, favoring works that explore profound...



## 10. Adding New Users to Population

Let's demonstrate how to add new users to an existing population and visualize them.

## 11. Saving and Loading Data

Let's demonstrate how to save and load our population data.

In [34]:
# Save the population
print("💾 Saving population to JSON...")
synthetic_population.save_to_json('demo_population.json')

print("✅ Population saved!")

# Also save in legacy format for compatibility
print("💾 Saving in legacy format...")
synthetic_population.export_embeddings_only('demo_population_legacy.json')

print("✅ Legacy format saved!")

2025-08-15 01:31:13,014 - INFO - Saved population 'Generated Population (200 users)' with 207 users to demo_population.json


💾 Saving population to JSON...
✅ Population saved!
💾 Saving in legacy format...
✅ Legacy format saved!


In [ ]:
# Load the population back
print("📁 Loading population from JSON...")
loaded_population = Population.load_from_json('demo_population.json')

print(f"✅ Loaded: {loaded_population}")
print(f"\n📊 Loaded Population Statistics:")
stats = loaded_population.get_statistics()
for key, value in stats.items():
    print(f"  {key}: {value}")

# Verify data integrity
original_users = len(synthetic_population.users)
loaded_users = len(loaded_population.users)

print(f"\n🔍 Data integrity check:")
print(f"  Original users: {original_users}")
print(f"  Loaded users: {loaded_users}")
print(f"  Match: {'✅' if original_users == loaded_users else '❌'}")

## 12. Population Analysis

Let's analyze our population using pandas DataFrames.

In [ ]:
# Convert population to DataFrame for analysis
df = synthetic_population.to_dataframe()

print("📊 Population DataFrame:")
print(df.head())

print(f"\n📈 DataFrame shape: {df.shape}")
print(f"\n📋 Columns: {list(df.columns)}")

In [ ]:
# Analyze the data
print("📊 Population Analysis:")
print(f"\n📚 Book Statistics:")
print(f"  Average books per user: {df['num_books'].mean():.1f}")
print(f"  Min books: {df['num_books'].min()}")
print(f"  Max books: {df['num_books'].max()}")

if 'avg_book_rating' in df.columns:
    avg_ratings = df['avg_book_rating'].dropna()
    if len(avg_ratings) > 0:
        print(f"\n⭐ Rating Statistics:")
        print(f"  Average rating: {avg_ratings.mean():.2f}")
        print(f"  Min average rating: {avg_ratings.min():.2f}")
        print(f"  Max average rating: {avg_ratings.max():.2f}")

print(f"\n🧠 Profile Completion:")
print(f"  Users with taste profiles: {df['has_taste_profile'].sum()}/{len(df)}")
print(f"  Users with embeddings: {df['has_embedding'].sum()}/{len(df)}")

## 13. Advanced Features

Let's demonstrate some advanced population features.

In [ ]:
# Split population for train/test
if len(synthetic_population) > 4:
    print("🔀 Splitting population into train/test sets...")
    
    train_pop, test_pop = synthetic_population.split_population(ratio=0.7, shuffle=True)
    
    print(f"✅ Train population: {train_pop}")
    print(f"✅ Test population: {test_pop}")
    
    print(f"\n📊 Split Statistics:")
    print(f"  Original: {len(synthetic_population)} users")
    print(f"  Train: {len(train_pop)} users ({len(train_pop)/len(synthetic_population)*100:.1f}%)")
    print(f"  Test: {len(test_pop)} users ({len(test_pop)/len(synthetic_population)*100:.1f}%)")
else:
    print("⚠️ Need more users to demonstrate population splitting")

In [ ]:
# Demonstrate merging populations
if 'train_pop' in locals() and 'test_pop' in locals():
    print("🔗 Demonstrating population merging...")
    
    # Create a new population
    merged_pop = Population("Merged Population")
    
    # Merge train and test populations
    merged_pop.merge_population(train_pop)
    merged_pop.merge_population(test_pop)
    
    print(f"✅ Merged population: {merged_pop}")
    print(f"📊 Original: {len(synthetic_population)}, Merged: {len(merged_pop)}")
    print(f"Match: {'✅' if len(synthetic_population) == len(merged_pop) else '❌'}")

## 14. Summary and Statistics

Let's wrap up with a comprehensive summary of what we've accomplished.

In [ ]:
# Final statistics
print("🎯 DEMO COMPLETE! 🎯")
print("=" * 50)

print(f"\n👥 Population Summary:")
print(f"  Name: {synthetic_population.name}")
print(f"  Total users: {len(synthetic_population)}")
print(f"  Users with embeddings: {len(synthetic_population.get_users_with_embeddings())}")
print(f"  Users with taste profiles: {len(synthetic_population.get_users_with_taste_profiles())}")

print(f"\n🤖 AI Usage Statistics:")
print(f"  Profile Generator: {profile_gen}")
print(f"  Embedding Generator: {embedding_gen}")

profile_stats = profile_gen.get_statistics()
embedding_stats = embedding_gen.get_statistics()

print(f"\n📊 API Usage:")
print(f"  Profile generation requests: {profile_stats['total_requests']}")
print(f"  Embedding generation requests: {embedding_stats['total_requests']}")
print(f"  Total API calls: {profile_stats['total_requests'] + embedding_stats['total_requests']}")

if 'visualizer' in locals():
    viz_stats = visualizer.get_method_statistics()
    print(f"\n📈 Visualization:")
    print(f"  Fitted methods: {viz_stats.get('fitted_methods', [])}")
    if 'pca_3d_explained_variance' in viz_stats:
        print(f"  PCA 3D explained variance: {viz_stats['pca_3d_explained_variance']:.3f}")

print(f"\n💾 Files Generated:")
generated_files = [
    'demo_population.json',
    'demo_population_legacy.json',
    'pca_2d_visualization.html',
    'umap_3d_visualization.html',
    'population_with_new_user.html'
]

for file in generated_files:
    if Path(file).exists():
        print(f"  ✅ {file}")
    else:
        print(f"  ❌ {file}")

print(f"\n🎉 Successfully demonstrated:")
demo_features = [
    "✅ User creation and management",
    "✅ Synthetic user generation with AI", 
    "✅ Taste profile generation",
    "✅ Embedding generation",
    "✅ Population management",
    "✅ Dimensionality reduction (PCA, t-SNE, UMAP)",
    "✅ Interactive visualization",
    "✅ Similarity search",
    "✅ Adding new users to populations",
    "✅ Data saving and loading",
    "✅ Population analysis"
]

for feature in demo_features:
    print(f"  {feature}")

print(f"\n🚀 The Mello ML Pipeline is ready for production!")